-----

## Setup

In [1]:
use Math::Matrix;
use Math::Splines;
use Math::Polynomial::Chebyshev;
use Math::Fitting;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::Importers;

use JavaScript::D3;
use JavaScript::Google::Charts;

use Hash::Merge;
use LLM::Configurations;

use Mathematica::Serializer;

### Google Charts

In [ ]:
#% javascript
google.charts.load('current', {'packages':['corechart']});
google.charts.load('current', {'packages':['gauge']});
google.charts.load('current', {'packages':['wordtree']});
google.charts.load('current', {'packages':['geochart']});
google.charts.load('current', {'packages':['table']});
google.charts.load('current', {'packages':['line']});
google.charts.setOnLoadCallback(function() {
    console.log('Google Charts library loaded');
});


#### Dark mode

In [ ]:
my $format = 'html';
my $titleTextStyle = { color => 'Ivory', fontSize => 16 };
my $backgroundColor = '#1F1F1F';
my $legendTextStyle = { color => 'Silver' };
my $legend = { position => "none", textStyle => {fontSize => 14, color => 'Silver'} };

my $hAxis = { title => 'x', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False, format => 'decimal'};
my $vAxis = { title => 'y', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False, format => 'decimal'};

my $annotations = {textStyle => {color => 'Silver', fontSize => 10}};
my $chartArea = {left => 50, right => 50, top => 50, bottom => 50, width => '90%', height => '90%'};

----

## Basic basis plots

### Single polynomial

In [ ]:
my @knots = b-spline-knots(n => 10, d => 3);
my &bFunc = b-spline-basis(:3degree, :@knots, :3index, arg => Whatever);

Single polynomial plot using a [Line chart](https://developers.google.com/chart/interactive/docs/gallery/linechart):

In [ ]:
#%html
my $n = 6;
my @data = (0, 1/100 ... 1).map({ [$_, &bFunc($_)] });
js-google-charts('LineChart', @data, 
    title => "B-spline basis", 
    :$titleTextStyle, :$backgroundColor, :$chartArea, :$hAxis, :$vAxis,
    width => 800, 
    div-id => 'poly1', :$format,
    :png-button)

### Full basis

Doing fitting we are interested in using bases of functions. Here for first eight Chebyshev-T polynomials make plot data:

In [ ]:
my $d = 3;
my @knots = b-spline-knots(:$d, n => 10);
my @basis = (^(@knots.elems - $d - 1)).map({ b-spline-basis(:$d, :@knots, index => $_, arg => Whatever) });
@basis.elems

In [ ]:
@basis.pairs.map({ $_.key.Str => $_.value.(0.5)})

In [ ]:
my @data = (0, 0.01 ... 1).map(-> $x { [x => $x, |@basis.pairs.map({ $_.key.Str => $_.value.($x) }) ].flat.Hash });

deduce-type(@data):tally;

Here is the plot with all eight functions:

In [ ]:
#%html
js-google-charts('LineChart', @data,
    column-names => ['x', |(^@basis.elems)».Str],
    title => "B-spline basis, 0 .. {@basis.elems - 1}",
    :$titleTextStyle,
    width => 800, 
    height => 400,
    :$backgroundColor, :$hAxis, :$vAxis,
    legend => merge-hash($legend, %(position => 'right')),
    chartArea => merge-hash($chartArea, %(right => 100)),
    format => 'html', 
    div-id => "cheb$n",
    :$format,
    :png-button)

-----

## Temperature data

Let us redo the whole workflow with a real life data -- weather temperature data for 4 consecutive years of Greenville, South Carolina, USA. 
(Where the [Perl and Raku Conference 2025](https://www.perl.com/article/get-ready-for-the-2025-perl-and-raku-conference/) is going to be held.)

Here we ingest the time series data:

In [ ]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/dsTemperature-Greenville-SC-USA.csv';
my @dsTemperature = data-import($url, headers => 'auto');
@dsTemperature = @dsTemperature.deepmap({ $_ ~~ / ^ \d+ '-' / ?? DateTime.new($_) !! $_.Num });
deduce-type(@dsTemperature)

Show data summary:

In [ ]:
sink records-summary(@dsTemperature, field-names => <Date AbsoluteTime Temperature>)

Here is a plot:

In [ ]:
#% html
js-google-charts("Scatter", @dsTemperature.map(*<Date Temperature>), 
    title => 'Temperature of Greenville, SC, USA',
    :$backgroundColor,
    hAxis => merge-hash($hAxis, {title => 'Time', format => 'M/yy'}), 
    vAxis => merge-hash($hAxis, {title => 'Temperature, ℃'}), 
    :$titleTextStyle, :$chartArea,
    width => 1200, 
    height => 400, 
    div-id => 'tempData', :$format,
    :png-button)

Here is a fit -- note the rescaling:

In [ ]:
my ($min, $max) = @dsTemperature.map(*<AbsoluteTime>).Array.&{ (.min, .max) }();

In [ ]:
my &rescale-time = { - $min / ($max - $min) + $_ / ($max - $min)};

my $d = 3;
my @knots = b-spline-knots(:$d, n => 14);
my @basis = (^(@knots.elems - $d - 1)).map({ b-spline-basis(:$d, :@knots, index => $_, arg => Whatever) o &rescale-time });

@basis.elems

In [ ]:
my &lm-temp = linear-model-fit(@dsTemperature.map(*<AbsoluteTime Temperature>), :@basis)

Her is a plot of the time series and the fit:

In [ ]:
my @fit = @dsTemperature.map(*<AbsoluteTime>)».&lm-temp;
my @plotData = transpose([@dsTemperature.map({ $_<AbsoluteTime> }).Array, @dsTemperature.map(*<Temperature>).Array, @fit]);
@plotData = @plotData.map({ <x data fit>.Array Z=> $_.Array })».Hash;

deduce-type(@plotData)

In [ ]:
#% html

my @ticks = @dsTemperature.map({ %( v => $_<AbsoluteTime>, f => $_<Date>.Str.substr(^7)) })».Hash[0, 120 ... *];

js-google-charts('ComboChart', 
    @plotData,
    title => 'Temperature data and Least Squares fit',
    column-names => <x data fit>,
    :$backgroundColor, :$titleTextStyle,
    hAxis => merge-hash($hAxis, {title => 'Time', :@ticks, textPosition => 'in'}), 
    vAxis => merge-hash($hAxis, {title => 'Temperature, ℃'}), 
    seriesType => 'scatter',
    series => {
        0 => {type => 'scatter', pointSize => 3, opacity => 0.1, color => 'Gray'},
        1 => {type => 'line', lineWidth => 4}
    },
    legend => merge-hash($legend, %(position => 'bottom')),
    :$chartArea,
    width => 1200, 
    height => 400, 
    div-id => 'tempDataFit', :$format,
    :png-button)